In [4]:
!pip install pythainlp python-crfsuite

     |████████████████████████████████| 743 kB 5.5 MB/s 


# Lao Part-of-speech

I use Yunshan Cup 2020 to train Lao Part-Of-sppech from https://github.com/FoVNull/SeqLabeling.

In [5]:
!wget https://github.com/GKLMIP/Yunshan-Cup-2020/raw/main/train.txt
!wget https://github.com/GKLMIP/Yunshan-Cup-2020/raw/main/dev.txt
!wget https://github.com/GKLMIP/Yunshan-Cup-2020/raw/main/test.txt

--2022-02-04 13:45:57--  https://github.com/GKLMIP/Yunshan-Cup-2020/raw/main/train.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GKLMIP/Yunshan-Cup-2020/main/train.txt [following]
--2022-02-04 13:45:58--  https://raw.githubusercontent.com/GKLMIP/Yunshan-Cup-2020/main/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1496147 (1.4M) [text/plain]
Saving to: ‘train.txt.1’

train.txt.1         100%[===================>]   1.43M  --.-KB/s    in 0.05s   

2022-02-04 13:45:58 (27.3 MB/s) - ‘train.txt.1’ saved [1496147/1496147]

--2022-02-04 13:45:58--  https://github.com/GKLMIP/Yunshan-Cup

In [6]:
import pickle
from pythainlp.tag import PerceptronTagger
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report

In [7]:
alltag = []

In [60]:
def readfile(name):
    with open(name ,"r",encoding='utf-8-sig') as f:
        data = [i.strip()  for i in f.readlines()]
    _list = []
    _p=[]
    for i in data:
      if i== '':
        _list.append(_p)
        _p=[]
      else:
        _w = tuple(i.split(' '))
        if len(_w)==2:
          _p.append(_w)
    return _list

In [61]:
train_dataset = readfile("train.txt")

In [62]:
train_dataset[-1]

[('ຈໍາ', 'N'),
 ('ເປັນ', 'V'),
 ('ແຫລະ', 'N'),
 ('ແນວ', 'N'),
 ('ບັນດາ', 'PRE'),
 ('ເຈົ້ານາຍ', 'N'),
 ('ບໍ່ໄດ້', 'NEG'),
 ('ຜ່ານທາງ', 'ADJ'),
 ('ນັ້ນ', 'DMN'),
 ('ອົດທົນ', 'V')]

In [63]:
valid_dataset = readfile("dev.txt")

In [64]:
valid_dataset[0]

[('ເປັນ', 'V'),
 ('ຄໍາ', 'N'),
 ('ແນະ', 'N'),
 ('ນໍາ', 'N'),
 ('ທີ່ດີ', 'ADJ'),
 ('ອ້າຍ', 'PRE'),
 ('ສົມ', 'PRN'),
 ('ຈິດ', 'PRN'),
 ('..', 'PUNCT')]

In [65]:
alltag = list(set(alltag))

In [66]:
p=0
for j in train_dataset:
  for i in j:
    try:
      (i[0],i[1])
    except:
      print(i)
      print(p)
    p+=1

In [67]:
p=0
for j in valid_dataset:
  for i in j:
    try:
      (i[0],i[1])
    except:
      print(i)
      print(p)
    p+=1

In [68]:
valid_x = [[i[0] for i in j] for j in valid_dataset]
valid_y = [[i[1] for i in j] for j in valid_dataset]

In [69]:
test_dataset = readfile("test.txt")

In [70]:
filename = "ptagger_Yunshan-Cup_corpus.json"

In [71]:
tagger = PerceptronTagger()

In [72]:
tagger.train(train_dataset, save_loc=filename)

In [73]:
tagger.tag(valid_x[0])

[('ເປັນ', 'V'),
 ('ຄໍາ', 'N'),
 ('ແນະ', 'N'),
 ('ນໍາ', 'PRN'),
 ('ທີ່ດີ', 'ADJ'),
 ('ອ້າຍ', 'V'),
 ('ສົມ', 'N'),
 ('ຈິດ', 'PRN'),
 ('..', 'PUNCT')]

In [74]:
valid_y[0]

['V', 'N', 'N', 'N', 'ADJ', 'PRE', 'PRN', 'PRN', 'PUNCT']

## check val

In [75]:
pred = [[t for _,t in tagger.tag(sent)] for sent in valid_x]
gold = valid_y

In [76]:
for i in range(0,len(pred)):
    if len(pred[i]) != len(gold[i]):
        print(i)
        print(pred[i],gold[i])

In [77]:
len(pred)

1600

In [78]:
len(gold)

1600

In [79]:
alltag

['NTR', 'V', 'ADV', 'N', 'PRE', 'IAC', 'CLF', 'ADJ', 'NEG']

In [80]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [81]:
print(bio_classification_report(gold, pred))

              precision    recall  f1-score   support

         ADJ       0.85      0.78      0.81      1155
         ADV       0.92      0.86      0.89       843
         CLF       0.94      0.85      0.90       456
         CNM       0.80      0.75      0.78       152
         COJ       0.92      0.90      0.91      1241
         DAN       0.50      0.50      0.50         2
         DAQ       1.00      1.00      1.00         7
         DBQ       0.80      0.80      0.80        59
         DMN       1.00      1.00      1.00       236
         FIX       0.90      0.94      0.92       147
         IAC       0.97      0.97      0.97       164
         IAQ       1.00      1.00      1.00        15
         IBQ       1.00      1.00      1.00         3
           N       0.81      0.90      0.85      7302
         NEG       0.99      0.99      0.99       249
         NTR       0.98      0.96      0.97       190
         ONM       1.00      0.12      0.22         8
         PRA       0.91    

## check test

In [82]:
test_x = [[i[0] for i in j] for j in test_dataset]
test_y = [[i[1] for i in j] for j in test_dataset]

In [83]:
test_pred = [[t for _,t in tagger.tag(sent)] for sent in test_x]
test_gold = test_y

In [84]:
for i in range(0,len(test_pred)):
    if len(test_pred[i]) != len(test_gold[i]):
        print(i)
        print(test_pred[i],test_gold[i])

In [85]:
len(test_pred),len(test_gold)

(3000, 3000)

In [86]:
print(bio_classification_report(test_gold, test_pred))

              precision    recall  f1-score   support

         ADJ       0.86      0.78      0.82      2311
         ADV       0.93      0.86      0.89      1636
         CLF       0.93      0.88      0.91       767
         CNM       0.88      0.79      0.83       246
         COJ       0.93      0.91      0.92      2351
         DAN       0.33      0.33      0.33         3
         DAQ       1.00      1.00      1.00         7
         DBQ       0.92      0.81      0.86       165
         DMN       1.00      1.00      1.00       467
         FIX       0.92      0.96      0.94       254
         IAC       0.98      0.97      0.98       337
         IAQ       0.97      0.95      0.96        41
         IBQ       1.00      0.57      0.73        14
           N       0.81      0.90      0.85     13859
         NEG       0.99      0.98      0.98       491
         NTR       0.97      0.96      0.97       322
         ONM       0.25      0.14      0.18         7
         PRA       0.91    